In [1]:
import pandas as pd
import re
import math
import numpy as np

metadata = pd.read_csv('/mnt/d/airdata/metadata_with_station.csv')
df = pd.read_csv('/mnt/d/airdata/noaa_redownload/la_2017_2021_v2.csv')
pm = pd.read_csv('/mnt/d/airdata/pm/la_train.csv')



/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,94,95,96,97,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def clean_weather_data_columns(data, remove_cols):
    #TO-DO: Explore what the columns mean and what can be removed, 
    # and if columns with much missing data are important
    '''
    Removes irrelevant columns and columns with many missing values
    for historical weather dataset. 
    Args: 
        data: weather dataframe
        remove_cols: columns to remove
    Returns: 
        Cleaned dataframe
    '''
    data = data.drop(remove_cols, axis = 1)
    data = df.dropna(axis=1, thresh=0.7)
    return data

def convert_to_int(x, err, acceptable_quality):
    if str(x) in acceptable_quality:
        return acceptable_quality[0]
    try:
        return float(x)
    except ValueError:
        return err

In [4]:
remove_cols = [ 'SOURCE', 'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL']
df = clean_weather_data_columns(df, remove_cols)
df['DATE'] = pd.to_datetime(df['DATE'])

In [5]:
def clean_wind(wnd):
    '''
    Parses wind data string into relevant fields. Creates new columns in weather dataframe
    Removes samples with corrupted wind data, or missing wind data. 
    'Wind direction: 0-360 degs
    'Wind speed: m/s
    '''
    acceptable_quality = (0,1,4,5,9) #see docs
    missing_dir, missing_spd = 999, 9999
    direction,quality_dir,_,speed,quality_spd = list(map(lambda x: convert_to_int(x, missing_spd, acceptable_quality), wnd.split(",")))
    wind_dir = direction
    wind_speed = speed
    if quality_dir not in acceptable_quality or direction == missing_dir or direction == missing_spd:
        wind_dir = np.nan
    if quality_spd not in acceptable_quality or speed == missing_spd:
        wind_speed  = np.nan
    return wind_speed, wind_dir
    

def clean_ceiling_height_data(ceiling):
    '''
    The height above ground level (AGL) of the lowest cloud or obscuring
    phenomena layer aloft with 5/8 or more summation total sky cover,
    which may be predominantly opaque, or the vertical visibility into a
    surface-based obstruction. Unlimited = 22000.
    height (m) above ground level of lowest cloud (unlimited = 22000)
    '''
    acceptable_quality = (0,1,4,5,9) #see docs
    missing = 99999
    height,quality, *rest =  list(map(lambda x: convert_to_int(x, missing, acceptable_quality), ceiling.split(",")))
    if quality not in acceptable_quality or height == missing:
            height = np.nan
    return height

def clean_visibility_data(vis):
    '''The horizontal distance at which an object can be seen and identified. (meters)
    '''
    acceptable_quality = (0,1,4,5,9) #see docs
    missing = 99999
    height,quality, *rest = list(map(lambda x: convert_to_int(x, missing, acceptable_quality), vis.split(",")))
    if quality not in acceptable_quality or height == missing:
            height = np.nan
    return height

def clean_temperature_data(tmp): 
    '''Air Temperature data in C'''
    missing = 9999
    acceptable_quality = (0,1,4,5,9,'C','I','M','P','R','U') #see docs
    temperature,quality = list(map(lambda x: convert_to_int(x, missing, acceptable_quality), tmp.split(",")))
    if quality not in acceptable_quality or temperature == missing:
        temperature = np.nan
    return temperature/10

def clean_pressure_data(pressure): 
    '''The air pressure relative to Mean Sea Level (MSL).
    (Hectopascals)'''
    missing = 99999
    acceptable_quality = (0,1,4,5,9) #see docs
    pressure,quality = list(map(lambda x: convert_to_int(x, missing, acceptable_quality), pressure.split(",")))
    if quality not in acceptable_quality or pressure == missing:
        pressure = np.nan
    return pressure

def clean_dew_point_data(dew): 
    '''The temperature to which a given parcel of air must be cooled
     at constant pressure and water vapor content in order for saturation to occur. (C)'''
    missing = 9999
    acceptable_quality = (0,1,4,5,9,'C','I','M','P','R','U') #see docs
    dp,quality = list(map(lambda x: convert_to_int(x, missing, acceptable_quality), dew.split(",")))
    if quality not in acceptable_quality or dp == missing:
        dp = np.nan
    return dp

def clean_precipitation_data(rain): 
    '''episode of LIQUID-PRECIPITATION.
    - The quantity of time over which the LIQUID-PRECIPITATION was measured. (hours)
    - The depth of LIQUID-PRECIPITATION that is measured at the time of an observation. (mm)
    Note that there data contains AA1-AA3 fields for multiple precipitation events
    98% of samples do not have more than 1 event (AA2-AA4 are 98% nan), so these are ignored 
    TO-DO: Check above statement on total dataste
    To-DO: how to deal with nan data (no rain event)
    '''
    times, depths, to_drop = [], [], []
    missing_depth, missing_time = 9999, 99
    acceptable_quality = ('0','1','4','5','9','C','I','M','P','R','U') #see docs
    if type(rain) == float:
        return np.nan, np.nan
    time,depth,_,quality = list(map(lambda x: convert_to_int(x, missing_depth, acceptable_quality), rain.split(",")))
    if quality not in acceptable_quality:
        depth = time = np.nan
    if depth == missing_depth or time == missing_time:
        depth = np.nan
    if time == missing_time or time == missing_depth:
        time = np.nan
    return time, depth


def clean(df):
    wind_speed, wind_dir = clean_wind(df['WND'])
    height = clean_ceiling_height_data(df['CIG'])
    vis = clean_visibility_data(df['VIS'])
    temperature = clean_temperature_data(df['TMP'])
    presssure = clean_pressure_data(df['SLP'])
    dew = clean_dew_point_data(df['DEW'])
    time, depth = clean_precipitation_data(df['AA1'])
    
    return wind_speed, wind_dir,height,vis,temperature,presssure,dew,time, depth
    
    
df[['wind_speed','wind_dir', 'ceiling', 'visibility', 'temperature', 'pressure', 'dew', 'duration', 'depth']] = df.apply(clean, axis=1, result_type ='expand')

In [6]:
cleaned_df = df[["STATION", "DATE", "LATITUDE", "LONGITUDE", "ELEVATION", 'wind_speed','wind_dir', 'ceiling', 'visibility', 'temperature', 'pressure', 'dew', 'duration', 'depth']].copy()

In [7]:
def average_wind_vector(speed, direction):
    '''
    Args: 
        wind_data: dataframe with wind direction and speed cols
    Returns: 
        tuple of mean x,y wind vectors
    '''
    #data is calibrated to north as 0 deg ?
    # wind_direction += 90
    # np.cos(dir + 90) * speed
    x_vect = np.cos(90 - direction) * direction
    y_vect = np.sin(90 - direction) * direction
    return np.nanmean(x_vect), np.nanmean(y_vect)


    
def average_cloud_height(cloud_height):
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    sigmoid = np.vectorize(sigmoid)
    cloud_height = sigmoid(cloud_height)
    height = np.nanmean(cloud_height)
    return height


In [8]:
def group_fnc(df):
    wind_x, wind_y = average_wind_vector(df['wind_speed'], df['wind_dir'])
    return pd.Series({
        "wind_x": wind_x, "wind_y":wind_y,
        "ceiling": np.nanmean(df["ceiling"]),
        "visibility": np.nanmean(df["visibility"]),
        "temperature": np.nanmean(df['temperature']),
        "pressure": np.nanmean(df['pressure']),
        "dew": np.nanmean(df['dew']),
        "precipitation_duration": np.nanmean(df['duration']),
        "precipitation_depth": np.nanmean(df['depth']),
        "latitude": df['LATITUDE'].iloc[0],
        'longtitude': df['LONGITUDE'].iloc[0],
    }) 

grouped_df = cleaned_df.groupby(['STATION', df['DATE'].dt.date]).apply(group_fnc).reset_index(drop=False)
grouped_df['DATE'] = pd.to_datetime(grouped_df['DATE'])

/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.
/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/home/ding/anaconda3/envs/COMP579_project/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]
/home/ding/anaconda3/envs/COMP579_pr

In [ ]:
grouped_df.to_csv('/mnt/d/airdata/noaa_redownload/la_weather_all_station_per_day.csv', index=False)

merge the pm dataset with the weather dataset

In [ ]:
pm_values = pm.merge(metadata, on='grid_id')[['STATION','day', 'pm25', 'mean_aod','min_aod','max_aod']]
pm_values['day'] = pd.to_datetime(pm_values['day'])
pm_values.rename(columns={'day':"DATE"}, inplace=True)


In [13]:
weather_with_pm = grouped_df.merge(pm_values, on=['STATION', 'DATE'])

In [14]:
weather_with_pm.merge(metadata, on='STATION').to_csv('/mnt/d/airdata/noaa_redownload/la_pm_per_day.csv', index=False)

In [1]:
import pandas as pd
import re
import math
import numpy as np

df = pd.read_csv('/mnt/d/airdata/noaa_redownload/la_2017_2021_v2.csv')



/home/ding/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,94,95,96,97,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['DATE'] = df['DATE'].dt.date

In [29]:
df.groupby(['STATION'])['DATE'].nunique().sort_values(ascending=False)

STATION
72295023174    1826
72391093111    1826
72288593197    1826
72288023152    1826
72295503174    1826
72295603167    1826
72297023129    1826
72297603166    1826
72288623130    1826
72297793184    1826
72392623136    1826
72287493134    1826
74704003102    1826
72286903171    1826
72286893138    1826
72286023119    1826
72033353175    1826
72289903179    1826
72286699999    1825
72297553141    1823
72292023191    1823
72292603154    1822
A0685400115    1816
74704303165    1816
99402899999    1815
99403599999    1812
74505753130    1811
72288703180    1802
72095299999    1566
72032999999    1305
99999923122    1235
72292800369    1138
72068799999     765
99999923122     587
72032999999     174
Name: DATE, dtype: int64

In [ ]:
train_labels = pd.read_csv('/mnt/d/airdata/train_labels.csv')
train_labels

In [34]:
train_labels['datetime'] = pd.to_datetime(train_labels['datetime'])

In [35]:
train_labels.groupby(['grid_id'])['datetime'].nunique().sort_values(ascending=False)

grid_id
3S31A    1047
PG3MI    1042
E5P9N    1040
ZZ8JF    1038
A2FBI    1036
QJHW4    1036
ZP1FZ    1035
WT52R    1034
H96P6    1034
DJN0F    1024
1X116     735
VR4WG     735
XJF9O     735
KW43U     735
9Q6TA     735
XNLVD     733
90BZ1     730
P8JA5     713
A7UCQ     709
90S79     684
HM74A     675
6EIL6     660
YHOPV     648
GVQXS     642
8KNI6     639
AZJ0Z     625
S77YN     618
D72OT     616
NE7BV     609
VYH7U     606
D7S1G     569
1Z2W7     569
KZ9W9     568
GAC6R     565
GJLB2     565
E2AUK     565
HANW9     556
CPR0W     545
IUMEZ     537
UC74Z     528
VXNN3     527
7334C     525
SZLMT     520
PW0JT     520
78V83     510
C7PGV     509
ZF3ZW     504
DHO4M     360
VBLD0     180
PJNW1     150
WZNCR     130
7F1D1     124
FRITQ     123
X5DKW     115
Name: datetime, dtype: int64